# **Graph creator**

A graph is a relativly natural way of representing molecules, and many method make use of structuring the data in this way.

This kernel shows a basic example of one can structure our data as a graph.   

Here we will create an array for our node values, and an adjacency matrix for our edge values. 


(**Note**: One can argue whether an adjacency matrix is really the best way to go here as we have an undirected graph, and it is therefore a bit innefficienct ( n^2 as opposed to n(n-1)/2 ), but it is easy to work with)

This is by no means the fastest way of doing this, but it is straightforward and only has to be run once, and the output can then be used. 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.spatial import distance_matrix

from sklearn import preprocessing
import os
print(os.listdir("../data/raw/"))

datadir = "../data/raw/"

['train_cycles.csv', 'data.covs.pickle', 'test_bonds.csv', '.ipynb_checkpoints', 'test.csv', 'mulliken_charges.csv', 'train.csv', 'structures.zip', 'train_charges.csv', 'structures_yukawa.csv', 'train_bonds.csv', 'structures.csv', 'scalar_coupling_contributions.csv', 'test_giba.csv.gz', 'test_cycles.csv', 'dipole_moments.csv', 'magnetic_shielding_tensors.csv', 'test_charges.csv', 'potential_energy.csv', 'train_giba.csv.gz', 'sample_submission.csv', 'angles.csv']


In [2]:
from tqdm.auto import tqdm

## Read in train, test and structures files

In [3]:
train = pd.read_csv(datadir + 'train.csv')
test = pd.read_csv(datadir + 'test.csv')
structures = pd.read_csv(datadir + 'structures.csv')

## Read in bonds files
Taken from:   https://www.kaggle.com/asauve/dataset-with-number-of-bonds-between-atoms  
(thanks Alexandre Sauvé!)

In [4]:
train_bonds = pd.read_csv(datadir + 'train_bonds.csv')
test_bonds = pd.read_csv(datadir + 'test_bonds.csv')

## Reads from angles file
Taken from: https://www.kaggle.com/soerendip/calculate-angles-and-dihedrals-with-networkx
(thanks Rakete!)

In [5]:
angs = pd.read_csv(datadir + "angles.csv")

## Normalize targets so they have are centered around 0 and have max of 1, and one-hot encode coupling types

In [6]:
#scale_min  = train['scalar_coupling_constant'].min()
#scale_max  = train['scalar_coupling_constant'].max()
#scale_mid = (scale_max + scale_min)/2
#scale_norm = scale_max - scale_mid

#train['scalar_coupling_constant'] = (train['scalar_coupling_constant'] - scale_mid)/scale_norm

# One hot encoding gets  too big for Kaggle, let's try label
#train[['1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN']] =  pd.get_dummies(train['type'])
#test[['1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN']]  =  pd.get_dummies(test['type'])

le = preprocessing.LabelEncoder()
le.fit(['1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN'])
train['l_type'] = (le.transform(train['type']) + 1)/8.
test['l_type'] = (le.transform(test['type']) + 1)/8.

## Pre-process the structures by one-hot encoding the atom types, and normalize distances to have around max of 1

In [7]:
structures[['C', 'F' ,'H', 'N', 'O']] = pd.get_dummies(structures['atom'])
structures[['x', 'y', 'z']] = structures[['x', 'y', 'z']]/10.

## Process bonds

In [8]:
test_bonds['nbond'] = test_bonds['nbond']/3
train_bonds['nbond'] = train_bonds['nbond']/3

# Process angles

In [9]:
angs['dihedral'] = angs['dihedral']/np.pi
# Should I rather one-hot this?
angs['shortest_path_n_bonds'] = angs['shortest_path_n_bonds']/6.0
angs = angs.fillna(0)

## Find training and testing molecules, and split structrues into test and train. Then group by molecule


In [10]:
train_mol_names = train['molecule_name'].unique()
test_mol_names  = test['molecule_name'].unique()

train_structures = structures.loc[structures['molecule_name'].isin(train_mol_names)]
test_structures = structures.loc[structures['molecule_name'].isin(test_mol_names)]

train_struct_group = train_structures.groupby('molecule_name')
test_struct_group  = test_structures.groupby('molecule_name')

train_group = train.groupby('molecule_name')
test_group  = test.groupby('molecule_name')

train_bond_group = train_bonds.groupby('molecule_name')
test_bond_group  = test_bonds.groupby('molecule_name')

train_angs = angs.loc[angs['molecule_name'].isin(train_mol_names)]
test_angs = angs.loc[angs['molecule_name'].isin(test_mol_names)]

train_angs_group = train_angs.groupby('molecule_name')
test_angs_group  = test_angs.groupby('molecule_name')

# Find max nodes in graph:
max_size = train_struct_group.size().max()

## Define node and edge values

In [11]:
# Values our nodes will have
node_vals = ['C', 'F' ,'H', 'N', 'O']#, 'x', 'y', 'z']
#Values our edges will have (minus distance, for now)
bond_vals = ['nbond']
j_coup_vals = ['l_type']#'1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN']
ang_vals = ['shortest_path_n_bonds','cosinus','dihedral']
edge_vals = j_coup_vals + bond_vals + ang_vals

# Find amount of training molecules
n_train_mols = len(train_mol_names)
n_test_mols = len(test_mol_names)

# Find dim of edges and nodes
bond_dim  = len(bond_vals)
j_coup_dim= len(j_coup_vals)
ang_dim   = len(ang_vals)
node_dim  = len(node_vals)
edge_dim  = len(edge_vals) 

# Additional edge dims for distances 
add_edge_dim = 1

## Pre-allocate arrays that we will fill later


In [17]:
train_nodes_array     = np.zeros((n_train_mols, max_size, node_dim), dtype=np.float32) 
train_in_edges_array  = np.zeros((n_train_mols, max_size, max_size, edge_dim + add_edge_dim),dtype=np.float32) 
train_out_edges_array = np.zeros((n_train_mols, max_size, max_size, 1),dtype=np.float32)
train_ids = np.zeros((n_train_mols, 135), dtype=np.int64)

test_nodes_array     = np.zeros((n_test_mols, max_size, node_dim), dtype=np.float32) 
test_in_edges_array  = np.zeros((n_test_mols, max_size, max_size, edge_dim + add_edge_dim),dtype=np.float32)
test_ids = np.zeros((n_test_mols, 135), dtype=np.int64)

In [13]:
def make_arrs(val_group, struct_group, bond_group, ang_group, test):
    i = 0
    for values, structs, bonds, angles in tqdm(zip(val_group, struct_group, bond_group, ang_group), total=len(val_group)):
        #if (not i%1000):
        #    print(i)

        # Calculate distances
        distances = np.zeros((max_size, max_size, 1))
        coords = structs[1][['x','y','z']].values
        dists  = distance_matrix(coords, coords)
        distances[:dists.shape[0],:dists.shape[1], 0] = dists 
        
        # Create nodes
        mol_info = structs[1][node_vals].values
        nodes = np.zeros((max_size, node_dim))
        nodes[:mol_info.shape[0], :mol_info.shape[1]] = mol_info

        # Create edges
        in_feats = np.zeros((max_size, max_size, j_coup_dim))
        ind = values[1][['atom_index_0', 'atom_index_1' ]].values
        in_feats[ind[:,0], ind[:,1], 0:j_coup_dim] = values[1][j_coup_vals].values
        in_feats[ind[:,1], ind[:,0], 0:j_coup_dim] = in_feats[ind[:,0], ind[:,1], 0:j_coup_dim]

        # Create bonds
        in_bonds = np.zeros((max_size, max_size, bond_dim))
        ind_bonds = bonds[1][['atom_index_0', 'atom_index_1' ]].values
        in_bonds[ind_bonds[:,0], ind_bonds[:,1]] = bonds[1][bond_vals].values
        in_bonds[ind_bonds[:,1], ind_bonds[:,0]] = in_bonds[ind_bonds[:,0], ind_bonds[:,1]]
        
        # Create angles
        ind_angs = angles[1][['atom_index_0', 'atom_index_1' ]].values
        ang_mat  = np.zeros((max_size, max_size, ang_dim))
        ang_mat[ind_angs[:,0], ind_angs[:,1]]  = angles[1][ang_vals]
        ang_mat[ind_angs[:,1], ind_angs[:,0]]  = ang_mat[ind_angs[:,0], ind_angs[:,1]]
        
        # concat all edge values 
        in_edges = np.concatenate((in_feats, in_bonds, ang_mat, distances),axis=2)
        


        
        if not test:           
            out_edges = np.zeros((max_size, max_size, 1))
            out_edges[ind[:,0], ind[:,1], 0] = values[1]['scalar_coupling_constant' ].values
            out_edges[ind[:,1], ind[:,0], 0] = out_edges[ind[:,0], ind[:,1], 0]
        

            train_nodes_array[i]      = nodes
            train_in_edges_array[i]   = in_edges
            train_out_edges_array[i]  = out_edges
        else:
            test_nodes_array[i]      = nodes
            test_in_edges_array[i]   = in_edges
        i = i + 1


In [16]:
make_arrs(train_group, train_struct_group, train_bond_group, train_angs_group, test = False)

In [17]:
make_arrs(test_group, test_struct_group, test_bond_group, test_angs_group, test = True)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Save as numpy arrays

In [16]:
np.save("nodes_train.npy" , train_nodes_array)
np.save("in_edges_train.npy" , train_in_edges_array)
np.save("out_edges_train.npy" , train_out_edges_array)

np.save("nodes_test.npy" , test_nodes_array)
np.save("in_edges_test.npy" , test_in_edges_array)